# The fundamentals of whole-cell modelling: stochastic simulation with Julia

In this notebook we will construct a simple model of gene expression in Julia, step by step.

### What is Julia?

<img src="figures/Julia_ecosystem.svg" alt="Julia Ecosystem" width="900"/>

<sub>Figure adapted from [Roesch, E., Greener, J.G., MacLean, A.L. et al. Julia for biologists. Nat Methods 20, 655–664 (2023).](https://doi.org/10.1038/s41592-023-01832-z)</sub>

Julia is a relatively young programming language that has become very popular in numerical and scientific computing due to its simplicity and performance, and is increasingly more often used to analyse and model biological data and systems due to its active state-of-the-art package ecosystem (as shown in the figure above). It also boasts a rich infrastructure for scientific computing that approaches C/C++ speeds while being significantly more user-friendly. If you have used Python or Matlab before, you should find Julia's syntax very familiar. 

In this notebook we will introduce the package [Catalyst.jl](https://github.com/SciML/Catalyst.jl), which allows us to model chemical reaction networks within Julia. We also make use of [DifferentialEquations.jl](https://docs.sciml.ai/DiffEqDocs/stable/), which provides efficient solvers for most kinds of differential equations one is likely to encounter in scientific modelling.

You can download and install Julia from the [official website](https://julialang.org/). 

### Setting up the environment

Julia's built-in package manager allows us to download and install packages from within Julia. 

**If you already use Julia:** To avoid situations where different projects require different, incompatible package versions, we recommend running this notebook inside its own environment. The following code will create a new environment in the project directory and install the packages we will need for this project:

In [ ]:
using Pkg             # Load the package manager

# ONLY RUN THIS IF YOU HAVE DIFFERENT VERSIONS OF THE PACKAGES BELOW INSTALLED
# This creates a new environment in the current working directory
#
# Pkg.activate(".")

# Install the packages we need
Pkg.add([ "Catalyst", "DifferentialEquations", "Plots", "Random", "Distributions" ])

Unlike Python, but like R, Julia frequently needs to precompile its packages. This may take a while, so we suggest installing all packages before the workshop starts.

**Note:** Catalyst.jl is currently in active development. To make use of all of its features we recommend running this notebook using Catalyst v14.

## A simple model of gene expression

We load packages with the `using` keyword, analogous to Python's `import`:

In [ ]:
using Catalyst

Let us start with a very simple example where we model the production and degradation of mRNA (which we call $M$) for a single gene. We assume that $M$ is produced at a constant rate $\rho_m$ and degraded with rate $\delta_m$. The network can be visualised schematically as follows:

![Birth-death model](figures/model_Poisson.svg)

We implement this in Catalyst using the `@reaction_network` macro\*, which takes a list of reactions and converts them into a reaction network in Julia.

\* A macro manipulates source code to introduce custom syntax into Julia. In this case, the macro `@reaction_network` allows us to specify a reaction network as a list of equations, which then get parsed and converted into ordinary Julia objects.

In [ ]:
rn = @reaction_network begin
    ρ_m, G --> G + M    # mRNA production
    δ_m, M --> 0        # mRNA degradation
end

 Catalyst automatically figures out which variables correspond to species and which ones to parameters. <!--If we want to explicitly specify these, we can write -->
<!-- ```julia
@species M(t)
@parameters ρ_m δ_m
```
within the reaction network. The notation `M(t)` tells us that $M$ is a time-dependent variable. -->

We can convert this network directly into an ordinary differential equation (ODE) describing how mRNA numbers change over time. This is represented as an `ODESystem` in the Catalyst ecosystem.

In [ ]:
osys = convert(ODESystem, rn)

As we can see, Catalyst took our symbolic description of the reaction network and converted it into a system of ODEs. If we want to simulate our system, we need to specify model parameters (the transcription rate $\rho_m$ and degradation rate $\delta_m$) and the initial conditions (what number of $G$ and $M$ molecules we start with).

In [ ]:
# Model parameters are given as a list of parameter-value pairs
# The notation :var indicates that 'var' is the name of a variable.
p = [ :ρ_m => 5, 
      :δ_m => 0.5 ]

# Initial conditions are given as a list of variable-initial value pairs
u0 = [ :G => 1, :M => 0 ]

# Simulation length
tmax = 10.;

The package DifferentialEquations.jl allows us to numerically solve these ODEs. For this purpose we set up an `ODEProblem` with our equation and the chosen parameters and initial values. DifferentialEquations takes care of choosing the right numerical solvers, tolerances and step sizes, so a simple call to `solve` is all that is necessary to solve our ODE problem.

In [ ]:
using DifferentialEquations

prob = ODEProblem(rn, u0, (0, tmax), p)
sol = solve(prob);

We can access the solution variables using bracket notation, as if the solution was a dictionary. Here the colon `:` is used to mark $M$ as the name of a variable, like a string. We used the same notation above when we specified parameters and initial values. 

In [ ]:
sol[:M]         # Returns an array

The solution times are given by `sol.t`:

In [ ]:
sol.t           # Another array

We can then easily [plot](https://docs.sciml.ai/DiffEqDocs/stable/basics/plot/) the obtained solution for $M$ using [Plots.jl](https://github.com/JuliaPlots/Plots.jl)

In [ ]:
using Plots

plot(sol.t, sol[:M], 
     xlabel="Time", 
     ylabel="Molecules",
     label="mRNA")

In the above plot, we notice that mRNA numbers behave weirdly - at places we have $0.3$, $1.8$ or $3.3$ molecules! This is clearly not realistic.

The problem is that ordinary differential equations treat `M` as a continuous variable that can take on any value. While this is a good approximation when mRNA numbers are very large (so they can take on many different values), this does not seem appropriate here. In reality we want mRNA numbers to be discrete, that is, to increase by $1$ whenever transcription happens, and decrease by $1$ whenever degradation occurs. To achieve this we treat our system as a jump differential equation, where jumps (transcription or degradation) happen at random times.

Constructing a `JumpProblem` is almost as easy as an `ODEProblem`. Again we need the reaction network, initial conditions and parameters.

In [ ]:
jinp = JumpInputs(rn, u0, (0, tmax), p)
prob = JumpProblem(jinp);

We solve this equation as before, using the `solve` function. Again, DifferentialEquations takes care of choosing the correct solver for us. In this case it uses the Stochastic Simulation Algorithm (SSA), also known as the [Gillespie algorithm](https://en.wikipedia.org/wiki/Gillespie_algorithm).

In [ ]:
sol = solve(prob);

Plotting the solution we see that mRNA numbers are now discrete, as they should be.

In [ ]:
plt = plot(sol.t, sol[:M], 
           xlabel="Time", 
           ylabel="Molecules",
           label="mRNA", 
           linetype=:steppost)

The result of the above simulation is stochastic: if we run `solve` again, we get a different trajectory. For example, we can generate and plot another two stochastic trajectories:

In [ ]:
for i in 1:2
    sol = solve(prob)
    plot!(sol.t, sol[:M], linetype=:steppost, label="")
end

plt

Try rerunning the cell above multiple times to add extra trajectories to the plot — just by simple visual expection we can gain intuition about the molecule count statistics, such as its mean and variance.

To obtain an deeper understanding of the stochastic dynamics of a system, we are often interested in computing the probability distribution of molecule counts at certain simulation times. This can be estimated by generating many SSA trajectories and creating a histogram of mRNA numbers:

In [ ]:
# Array to store the result of each simulation
ms = []
# Number of simulations to perform
n_trajectories = 1000

for i in 1:n_trajectories
    sol = solve(prob)
    m = sol(tmax, idxs=:M) # record the mRNA count at time t = tmax
    push!(ms, m)
end

# Compute and plot the discrete probability distribution
histogram(ms, 
          bins=0:maximum(ms), # define bins from 0 to the maximum observed mRNA count
          normalize=:probability,
          xlabel="Molecules", 
          ylabel="Probability", 
          label="mRNA") 

### Exercises:
* What effect does changing $\rho_m$ have on the trajectories of the system? What about $\delta_m$?
* By running $1000$ (or more) simulations, can you estimate the average number of mRNA and its variance?
* Try estimating the mRNA distribution at different simulation times. How does the distribution and its moments evolve in time?

## Gene regulation

![Telegraph model](figures/telegraph_model.svg)

Most genes in real cells are regulated: they can be transcriptionally active (on) or inactive (off). We model this by separating $G$ into two species, $G_{on}$ and $G_{off}$, and now allow switching between the states with rates $\sigma_{on}$ and $\sigma_{off}$ — this model of gene expression is also known as the telegraph model.

In [ ]:
rn = @reaction_network begin
    σ_on, G_off --> G_on      # gene activation
    σ_off, G_on --> G_off     # gene inactivation
    
    ρ_m, G_on --> G_on + M    # mRNA transcription
    δ_m, M --> 0              # mRNA degradation
end

We have to update our parameters and initial conditions:

In [ ]:
# Model parameters
p = [ :σ_on => 1,
      :σ_off => 1,
    
      :ρ_m => 5, 
      :δ_m => 0.5 ]

# Start with the gene in the off state
u0 = [ :G_off => 1, :G_on => 0, :M => 0 ];

As before we can convert this into a jump differential equation and solve it:

In [ ]:
jinp = JumpInputs(rn, u0, (0, tmax), p)
prob = JumpProblem(jinp);

In [ ]:
sol = solve(prob);

In [ ]:
plot(sol.t, sol[:M], 
     xlabel="Time", 
     ylabel="Molecules",
     label="M", 
     linetype=:steppost)

plot!(sol.t, 
      sol[:G_on], 
      label="G_on", 
      linetype=:steppost)

Here the `plot!` function overwrites the previous plot, so that we can observe both mRNA activity and the gene state in one figure. Note that mRNA numbers only increase when the gene is active (but they can decrease at any time).

### Exercises:
* Can you estimate the probability that the gene is on at time $t = 10$? How does changing the switching rates $\sigma_{on}$ and $\sigma_{off}$ affect this?
* Why did we introduce two species, $G_{on}$ and $G_{off}$? Consider what happens if you model the system using one species $G$ with the following reactions for activation and inactivation:

$$ 0 \longrightarrow G $$
$$ G \longrightarrow 0 $$

## Adding protein

![Telegraph model with protein translation](figures/telegraph_model_with_translation.svg)

This section is an exercise. The goal is to add protein to the previous reaction system. We assume that the protein (which we denote $P$) is translated from mRNA at rate $\rho_p$, and degraded with rate $\delta_m$. Add these two reactions to the previous reaction network, simulate the system and plot the solution. You can use the parameter values $\rho_p = 10$ and $\delta_p = 0.1$, or come up with your own. Play around with parameter values and initial conditions to see how the system behaves!

## Feedback

![Feedback loop](figures/feedback_model.svg)

Our previous model of gene expression is very simplistic, in that the gene switches on and off at random times without us specifying exactly how that happens. In reality, this is a very deliberaten process. Many genes self-regulate, that is, they promote or repress their own transcription. We can modify our previous system to model a positive feedback loop, by assuming that our protein $P$ binds to its own transcription start site and prevents deactivation of the gene. We implement this by incorporating the protein concentration in the gene inactivation mechanism, using a repressive Michaelis-Menten equation (the function `mmr` in Catalyst):

In [ ]:
rn = @reaction_network begin
    σ_on, G_off --> G_on      # gene activation
    
    # gene inactivation now depends on protein numbers
    mmr(P, σ_off, K), G_on --> G_off
    
    ρ_m, G_on --> G_on + M    # transcription
    δ_m, M --> 0              # mRNA degradation

    ρ_p, M --> M + P          # translation
    δ_p, P --> 0              # protein degradation
end

Here the gene inactivation rate is

$$ \sigma_{off} \cdot \frac{K}{K + P} $$

which is inhibiting for high $P$. For $P = 0$, this is simply $\sigma_{off}$, so nothing changes. As $P$ increases, the rate decreases; at concentration $K$, inactivation is only 50% as fast, and as $P$ increases beyond that, the reaction rate approaches $0$. 

In [ ]:
# Model parameters
p = [ :σ_on => 1,
      :σ_off => 1,
      :K => 40,
    
      :ρ_m => 5, 
      :δ_m => 0.5,

      :ρ_p => 10,
      :δ_p => 1 ]

# Start with the gene in the off state
u0 = [ :G_off => 1, :G_on => 0, :M => 0, :P => 0 ];

tmax = 50.;

As before we can convert this into a jump differential equation and solve it:

In [ ]:
jinp = JumpInputs(rn, u0, (0, tmax), p)
prob = JumpProblem(jinp);

In [ ]:
sol = solve(prob);

In [ ]:
plot(sol.t, sol[:M], 
     xlabel="Time", 
     ylabel="Molecules",
     label="M", 
     linetype=:steppost)

plot!(sol.t, 
      sol[:G_on], 
      label="G_on", 
      linetype=:steppost)

In [ ]:
plot(sol.t, sol[:P], 
     xlabel="Time", 
     ylabel="Molecules",
     label="P", 
     linetype=:steppost)

### Exercises
* What are other ways in which positive autoregulation could be built into this system? Think about examples of positive feedback you have encountered in biological systems.
* How could you incorporate negative feedback into this system? How does the system behave in this case?

## Responding to the environment

<img src="figures/lac_operon_1.svg" alt="lac operon" width="1000"/>

While some genes are expressed throughout the life cycle of a cell, others are only activated by certain signals. Many heat shock proteins (HSPs) are induced as a response to cellular stress, and some bacteria produce certain enzymes only when they detect the corresponding substrate. A very well studied example of this is the *lac* operon in *E. coli*, which cells can use to digest lactose when glucose is absent. In this section we will build a drastically simplified version of what happens in that operon.

The *lac* operon contains two genes involved in lactose metabolism: *lacZ*, which cleaves lactose into glucose and galactose, and *lacY*, which transports lactose into the cell. Expression of these genes is normally deactivated by another gene (*lacI*) that acts as a repressor and blocks the transcription of mRNA from the operon. Certain lactase metabolites bind to the repressor and deactivate it, which allows for the production of *lacY* and *lacZ*. 

The *lac* operon can be induced experimentally by the chemical IPTG (*Isopropyl β-D-thiogalactopyranoside*), which mimics a lactose metabolite and also activates the *lac* operon. We can induce *lac* expression in a controlled manner by introducing IPTG into the cell culture. We extend our previous model by adding another reaction that depends on the measured IPTG concentration and promotes gene deactivation. We assume that the extracellular IPTG concentration is described by the following function:

In [ ]:
function sigmoid(x)
    return 1 / (1 + exp(-x))
end

function iptg_conc(t)
    return 5e-4 * sigmoid(10t - 60) * sigmoid(120 - 10t)
end;

In [ ]:
xx = 0:0.1:50.             # Plot values from t = 0 to 50
yy = iptg_conc.(xx)        # The dot computes the lactose concentration for each value of x

plot(xx, yy, xlabel="Time", ylabel="Concentration", label="IPTG")

In this model, the protein (now named *lacZ*) metabolises lactose at a constant rate into allolactose, a sugar that promotes the *lac* operon much like IPTG. This is again a simplification, as we do not model lactose explicitly — in reality we would have to introduce the protein *lacY*, which transports lactose into the cell. We can justify our choice if we assume that lactose is already abundantly present in the cell, so that its concentration is effectively constant.

In [ ]:
rn = @reaction_network begin
    σ_on, G_off --> G_on      # gene activation
    
    # gene inactivation due to either IPTG or allolactase
    mmr(iptg_conc(t) + A / Ω, σ_off, K), G_on --> G_off
    
    ρ_m, G_on --> G_on + M    # transcription
    δ_m, M --> 0              # mRNA degradation

    ρ_p, M --> M + lacZ       # translation
    δ_p, lacZ --> 0           # protein degradation

    μ_a, lacZ --> lacZ + A    # allolactose production (ignore the lactos that gets spent)
    δ_a, A --> 0              # allolactose degradation
end

Here the gene inactivation rate is

$$ \sigma_{off} \cdot \frac{K}{K + [IPTG] + [A]} $$

which means that both $IPTG$ and allolactase $A$ have the same effect, of blocking deactivation of the gene. Here the square brackets denote concentrations. We compute the concentration of $A$ by dividing the number of molecules by the cell volume; we call the conversion factor $\Omega$. Here we do not model lactose explicitly - we can do this if we assume that its concentration stays effectively constant throughout the experiment.

In [ ]:
# Parameters. Note that these are highly biologically unrealistic
p = [ :σ_on => 1,
      :σ_off => 50,
    
      :K => 1e-5,
      :Ω => 2e5,
    
      :ρ_m => 2, 
      :δ_m => 2,

      :ρ_p => 2,
      :δ_p => 0.2,

      :μ_a => 100,
      :δ_a => 2 ]

u0 = [ :G_off => 1, :G_on => 0, :M => 0, :lacZ => 0, :A => 0 ];

tmax = 50.;

In [ ]:
jinp = JumpInputs(rn, u0, (0, tmax), p)
prob = JumpProblem(jinp);

Since we are now using a time-dependent reaction (via the IPTG concentration), we cannot use the SSA to solve the equations. As DifferentialEquations is unsure what solver to use in this case, we will use a standard ODE solver (`Tsit5`) for the time-dependent part; the jumps are still going to modelled via the SSA under the hood.

In [ ]:
sol = solve(prob, Tsit5());

In [ ]:
plot(sol.t, sol[:M], 
     xlabel="Time", 
     ylabel="Molecules",
     label="M", 
     linetype=:steppost)

plot!(sol.t, 
      sol[:G_on], 
      label="G_on", 
      linetype=:steppost)

In [ ]:
plot(sol.t, sol[:lacZ], 
     xlabel="Time", 
     ylabel="Molecules",
     label="P", 
     linetype=:steppost)

In [ ]:
plot(sol.t, sol[:A], 
     xlabel="Time", 
     ylabel="Molecules",
     label="P", 
     linetype=:steppost)

## Adding cell cycle dynamics

<img src="figures/lac_operon_1_cc.svg" alt="lac operon" width="1000"/>

Wholistic modelling of cells needs to incorporate the fact that they grow and divide. While we will ignore volume in this tutorial, we can implement the cell cycle in Catalyst by adding callbacks at times that our cell divides - by choosing one daughter cell at random at each division, we will simulate a lineage. For this we have to specify when this occurs (say, every 30 minutes) and what happens during division. In our case, we will divide mRNA, protein and metabolite numbers by half - more precisely, we assume that each molecule has a 50% chance of ending up in the each daughter cell.

In [ ]:
using Distributions

# The argument `current_sol` contains information about the current cell state
function divide!(current_sol)
    # mRNA, proteins and metabolites are divided randomly between the daughters
    # We can access variables using bracket notation
    current_sol[:M] = rand(Binomial(current_sol[:M], 0.5))
    current_sol[:lacZ] = rand(Binomial(current_sol[:lacZ], 0.5))
    current_sol[:A] = rand(Binomial(current_sol[:A], 0.5))
end;

T_cellcycle = 20.;

[Callbacks](https://docs.sciml.ai/DiffEqDocs/stable/features/callback_functions/) are very simple to implement in DifferentialEquations: a simple addition to our `solve` call will do the trick:

In [ ]:
sol = solve(prob, Tsit5(); callback=PeriodicCallback(divide!, T_cellcycle));

In the plots below we have marked the division times explicitly. While they are not easy to read off from the mRNA plots due to their volatility, they are more noticeable when looking at allolactose concentrations.

In [ ]:
plot(sol.t, sol[:M], 
     xlabel="Time", 
     ylabel="Molecules",
     label="M", 
     linetype=:steppost)

plot!(sol.t, 
      sol[:G_on], 
      label="G_on", 
      linetype=:steppost)

vline!([ T_cellcycle, 2T_cellcycle ], color=:black)

In [ ]:
plot(sol.t, sol[:lacZ], 
     xlabel="Time", 
     ylabel="Molecules",
     label="P", 
     linetype=:steppost)

vline!([ T_cellcycle, 2T_cellcycle ], color=:black)

In [ ]:
plot(sol.t, sol[:A], 
     xlabel="Time", 
     ylabel="Molecules",
     label="P", 
     linetype=:steppost)

vline!([ T_cellcycle, 2T_cellcycle ], color=:black)

## Outlook

The above was intended to give a glimpse into the world of biochemical modelling in Julia. For an example of a more realistic implementation of the *lac* operon, check out the supplementary notebook for this workshop. Beyond this, there are countless ways to build more realistic models including some of the aspects below:

### Whole cells

So far we only focussed on modelling one specific phenomenon in *E. coli*: the *lac* operon. This does not operate in isolation, but is coupled to metabolism, gene expression, signalling pathways and more in an intricate network of interactions. The ambitious goal of whole-cell modelling is to incorporate all of these into one model in order to understand how cells work on a wholistic level.

### Realistic cell cycle modelling

Cells undergo different phases (G1, S, G2/M etc.) during their lifetime, which we did not need to model explicitly in our simple model. Cell cycle progression and especially division do not occur at fixed times, but are controlled by complex molecular machinery that is still to be explored. Furthermore, division may be asymmetric (as in budding yeast), and partitioning of molecules may not be entirely binomial. A more realistic depiction could include these aspects instead of requiring division by half every 20 minutes.

### Cell size

Many intracellular processes including gene expression and metabolism are strongly affected by cell size, which we did not mention here. We could model volume $V$ by coupling the above reaction network to an ordinary differential equation of the form

$$ \frac{d}{dt} V = \gamma V $$

where $\gamma$ is the growth rate of a cell. Of course, during division the daughter cell inherits a fraction of the mother's volume. We could even make volume growth itself stochastic (turning this into a stochastic differential equation) and have cells divide once they reach a fixed volume - this approximates the cell cycle dynamics of fission yeast.

### Spatial modelling

Cells are not just little blobs in which stuff happens - many of them are very strictly organised inside. This is especially the case for Eukaryotes, which generally have a nucleus, a cytosol and countless organelles like mitochondria. A more realistic model of eukaryotic gene expression could distinguish e.g. the nucleus (in which mRNA transcription and folding happens) and the cytosol (where most proteins are translated). This is also important for complex signal pathways in which information has to be relayed from the cell surface to the nucleus, and vice versa.

### More cells

Cells like to communicate with each other. To model tissues and organisms, it is often necessary to combine many cells into one network of interactions, allowing them to exchange information and respond collectively to the environment. Catalyst now provides [functionality for spatial modelling](https://docs.sciml.ai/Catalyst/stable/spatial_modelling/lattice_reaction_systems/) that can be used to construct such models with relative ease.